<a href="https://colab.research.google.com/github/chawadesu/HandsOnDL/blob/main/YoLO_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Mar 18 01:34:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load ROBO FLOW DATA MaiOOK

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="xxx")
project = rf.workspace("xxx").project("xxx")
dataset = project.version(2).download("yolov5")

     |████████████████████████████████| 1.1 MB 10.7 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 145 kB 58.4 MB/s 
     |████████████████████████████████| 138 kB 75.4 MB/s 
     |████████████████████████████████| 596 kB 64.2 MB/s 
     |████████████████████████████████| 178 kB 75.5 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=a37f904bec166528f6bd9248f940251fd17b58a1328b0281f5980d7743b4bd03
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.7
    Uninstalling pyparsing-3.0.7:
      Successfully uninstalled pyparsing-3.0.7
 

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Properties-Tag-Equalized-v2-2 in yolov5pytorch:: 100%|██████████| 9427/9427 [00:01<00:00, 4771.71it/s]


In [ ]:
!pip install wandb
!wandb login xxx

     |████████████████████████████████| 1.7 MB 9.0 MB/s 
     |████████████████████████████████| 181 kB 22.2 MB/s 
     |████████████████████████████████| 144 kB 81.2 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=acf16c9e129c0fa24d1136f70e59376d10beeec092ddbb33d75af8f98763a123
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# YOLO INSTALL

In [ ]:
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 11371, done.
remote: Total 11371 (delta 0), reused 0 (delta 0), pack-reused 11371
Receiving objects: 100% (11371/11371), 11.28 MiB | 28.10 MiB/s, done.
Resolving deltas: 100% (7850/7850), done.
/content/yolov5
Setup complete. Using torch 1.10.0+cu111 (Tesla P100-PCIE-16GB)


In [ ]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

In [ ]:
%cd /content

/content


In [ ]:
!pip -q install --upgrade --no-cache-dir gdown

# Load Fine-Tune Model
# !gdown --id 1Ba3HQXAWnP3OlgbwsWV83D7hD2qyA-qE # v5m
!gdown --id xxx # v5m-finetune v2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1F_DOMPl7Mz8TO_hvl9s0vd2ps8mGxIwE
To: /content/yolo_v5m_finetune_v2.pt
100% 42.2M/42.2M [00:00<00:00, 65.9MB/s]


# Step 3: Train Our Custom YOLOv5 model

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset locaiton is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

In [ ]:
!wandb disabled
!wandb enabled

W&B disabled.
W&B enabled.


In [ ]:
!pwd

/content


In [ ]:
DATA_PATH = "/content/Properties-Tag-Equalized-v2-2/data.yaml"
WEIGHT = "/content/yolo_v5m_finetune_v2.pt"
AUG_PATH = "/content/yolov5/data/hyps/hyp.scratch-high.yaml"
!python /content/yolov5/train.py \
    --img 416 \
    --batch 64 \
    --epochs 200 \
    --data $DATA_PATH \
    --weights $WEIGHT \
    --cache \
    --patience 30 \
    --hyp $AUG_PATH
    # --save-period 5 \


wandb: Currently logged in as: huak95 (use `wandb login --relogin` to force relogin)
train: weights=/content/yolo_v5m_finetune_v2.pt, cfg=, data=/content/Properties-Tag-Equalized-v2-2/data.yaml, hyp=/content/yolov5/data/hyps/hyp.scratch-high.yaml, epochs=200, batch_size=64, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=30, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-38-g7c6a335 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup

In [ ]:
import glob

# Use newest path
last_train_path = sorted(glob.glob('/content/yolov5/runs/train/*'))[-1]
print("last_train_path: ", last_train_path)

last_train_path_weight = last_train_path+'/weights/best.pt'


last_train_path:  /content/yolov5/runs/train/exp


In [ ]:
output_weight_path = "/content/drive/Shareddrives/SuperAI_Dataset/Image_Classification/Model/"+ "v3_" + WEIGHT.split('/')[-1]

In [ ]:
!cp -r $last_train_path_weight $output_weight_path